In [14]:
import json
from shapely.geometry import Point, Polygon, MultiPolygon
import geojson
import geopandas as gpd
import pandas as pd
import numpy as np
from time import time
import leafmap.foliumap as leafmap
import supervision as sv
from garuda.od import ConfusionMatrix
from IPython.display import Markdown

In [15]:
region = 'bharatpur'
prediction_file = f"../predict/m1_obb_without_empty_train_obb_v1_640_100/{region}/predictions_0.25.geojson"
print(f"Region: {region}")
print(f"Prediction file: {prediction_file}")

Region: bharatpur
Prediction file: ../predict/m1_obb_without_empty_train_obb_v1_640_100/bharatpur/predictions_0.25.geojson


In [17]:
prediction_gdf = gpd.read_file(prediction_file) 
prediction_gdf['yolo_label'] = prediction_gdf['yolo_label'].apply(lambda x: np.array(list(map(float, x.split("_")))))
print(len(prediction_gdf))
print(prediction_gdf.crs)
prediction_gdf.head()



101
EPSG:3857


,x,y,class_name,confidence,yolo_label,geometry
0,8596722,3108844,FCBK,0.482616,"[1.0, 0.974679, 0.492202, 0.987097, 0.505247, ...","POLYGON ((8598173.407 3108868.268, 8598211.375..."
1,8585715,3139113,CFCBK,0.863236,"[0.0, 0.858775, 0.657242, 0.85979, 0.627291, 0...","POLYGON ((8586812.1 3138632.724, 8586815.204 3..."
2,8585715,3139113,FCBK,0.804194,"[1.0, 0.882969, 0.870271, 0.891651, 0.852002, ...","POLYGON ((8586886.073 3137981.392, 8586912.618..."
3,8585715,3139113,FCBK,0.586538,"[1.0, 0.416694, 0.41831, 0.434965, 0.425009, 0...","POLYGON ((8585460.446 3139363.254, 8585516.309..."
4,8585715,3139113,Zigzag,0.473407,"[2.0, 0.429403, 0.802783, 0.438607, 0.785262, ...","POLYGON ((8585499.304 3138187.735, 8585527.445..."


In [18]:
prediction_gdf.crs.to_string()
print(prediction_gdf.head(2))

         x        y class_name  confidence  \
0  8596722  3108844       FCBK    0.482616   
1  8585715  3139113      CFCBK    0.863236   

                                          yolo_label  \
0  [1.0, 0.974679, 0.492202, 0.987097, 0.505247, ...   
1  [0.0, 0.858775, 0.657242, 0.85979, 0.627291, 0...   

                                            geometry  
0  POLYGON ((8598173.407 3108868.268, 8598211.375...  
1  POLYGON ((8586812.1 3138632.724, 8586815.204 3...  


In [19]:
prediction_gdf.reset_index(inplace=True, drop=True)
print(prediction_gdf.head(2))

         x        y class_name  confidence  \
0  8596722  3108844       FCBK    0.482616   
1  8585715  3139113      CFCBK    0.863236   

                                          yolo_label  \
0  [1.0, 0.974679, 0.492202, 0.987097, 0.505247, ...   
1  [0.0, 0.858775, 0.657242, 0.85979, 0.627291, 0...   

                                            geometry  
0  POLYGON ((8598173.407 3108868.268, 8598211.375...  
1  POLYGON ((8586812.1 3138632.724, 8586815.204 3...  


In [20]:
intersection_gdf = gpd.sjoin(prediction_gdf, prediction_gdf, predicate="intersects")[['index_right']].reset_index()
intersection_gdf = intersection_gdf[intersection_gdf['index'] < intersection_gdf['index_right']]
len(intersection_gdf)

4

In [21]:
def get_iou(row):
    left_polygon = prediction_gdf.iloc[int(row['index'])]['geometry']
    right_polygon = prediction_gdf.iloc[int(row['index_right'])]['geometry']
    left_area = left_polygon.area
    right_area = right_polygon.area
    iou = left_polygon.intersection(right_polygon).area / left_polygon.union(right_polygon).area
    return iou, left_area, right_area

intersection_gdf[['iou', 'left_area', 'right_area']] = intersection_gdf.apply(get_iou, axis=1, result_type="expand")
intersection_gdf = intersection_gdf[intersection_gdf['iou'] > 0.33]
len(intersection_gdf)
print(intersection_gdf.head(2))

    index  index_right       iou    left_area   right_area
40     38           46  0.442287  3908.013548  8127.644234


In [22]:
drop_idx = intersection_gdf.apply(lambda x: x['index'] if x['left_area'] < x['right_area'] else x['index_right'], axis=1).astype(int).values
print(len(prediction_gdf))
prediction_gdf.drop(drop_idx, inplace=True)
print(len(prediction_gdf))
display(prediction_gdf.head(2))

101
100


,x,y,class_name,confidence,yolo_label,geometry
0,8596722,3108844,FCBK,0.482616,"[1.0, 0.974679, 0.492202, 0.987097, 0.505247, ...","POLYGON ((8598173.407 3108868.268, 8598211.375..."
1,8585715,3139113,CFCBK,0.863236,"[0.0, 0.858775, 0.657242, 0.85979, 0.627291, 0...","POLYGON ((8586812.1 3138632.724, 8586815.204 3..."


In [23]:
final_kilns=prediction_gdf.to_crs(epsg=4326)
display(final_kilns.head(2))

,x,y,class_name,confidence,yolo_label,geometry
0,8596722,3108844,FCBK,0.482616,"[1.0, 0.974679, 0.492202, 0.987097, 0.505247, ...","POLYGON ((77.23871 26.88305, 77.23905 26.88273..."
1,8585715,3139113,CFCBK,0.863236,"[0.0, 0.858775, 0.657242, 0.85979, 0.627291, 0...","POLYGON ((77.13665 27.12128, 77.13667 27.12202..."


In [24]:
import os
save_handvalidation_dir = f"../hand_validation/{region}"
os.makedirs(save_handvalidation_dir, exist_ok=True)

In [25]:
version = "v1"  
batch_size = 500
for i in range(0, len(final_kilns), batch_size):
    end = min(i+batch_size, len(final_kilns))
    final_kilns.iloc[i:i+batch_size].to_file(f"../hand_validation/{region}/{version}_potential_kiln_{i}_{end-1}.geojson", driver='GeoJSON')